<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Василий, привет! Спасибо, что прислал работу во время :) 
    
Меня зовут Евгений, и я буду проверять твой проект) 
Ты проделал большую работу над проектом, однако в любом деле всегда можно можно что-то улучшить. Моя основная цель — не показать, что где—то совершены какие—либо ошибки, а поделиться своим опытом, который поможет тебе в дальнейших работах. Далее в файле ты сможешь увидить мои комментарии, постарайся учесть их при выполнении следующих заданий.    

Мои комментарии обозначены пометкой **Комментарий ревьюера**. Далее в файле ты сможешь найти их в похожих ячейках (если фон комментария <font color=green>зелёный</font> - всё сделано правильно; <font color=orange>оранжевый</font> - есть замечания, но не критично; <font color=red>красный</font> - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения данного проекта. 

</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Рекомендуется делать краткое вступление в работу и описание того, что планируешь сделать (если план у тебя уже сформирован), а также информацию о входных данных: какие столбцы есть в таблице, их названия и какую информацию они несут. Так работа станет выглядеть презентабельнее.

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [6]:
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('data.csv')
m = Mystem()

Installing mystem to C:\Users\python_dev/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.0-win7-32bit.zip


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


In [8]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В датафрейме 21525 строк различного типа данных. 
Столбец days_employed имеет некорректные данные для общего трудового стажа в днях. 
В столбцах days_employed и total_income иимеются пропуски.

    От кредитного отдела банка была получена статистика платежеспособности клиентов. Задача выявить зависимости от семейного положения, наличия детей, дохода и целей кредита на факт погашения кредита в срок. Данные получены в виде файла .csv.
    В файле 21525 записей

### Шаг 2. Предобработка данных

Для ответа на поставленные вопросы проверяем данные в колонке о детях

In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Имеются аномальные данные о количестве детей: -1, 20(в большом отрыве)

In [10]:
print('Артефакты в данных о количестве детей составлют {:.1%}'.format((data.loc[data['children'] == 20]['children'].count()+data.loc[data['children'] == -1]['children'].count()) / data['children'].count())) 

Артефакты в данных о количестве детей составлют 0.6%


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Интересный способ посчитать доли в данных.
    
---
    
У метода `value_counts()` есть атрибут `normalize`, который принимает булевы значения. Этот атрибут позволяет посчитать доли в данных. 1
    
</div>

заполняем артефакты нулевым значением

In [11]:
data['children'].replace(-1 , 0, inplace=True)
data['children'].replace(20 , 0, inplace=True)
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Информация об образовании

In [12]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Информация о семейном положении

In [13]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Обработка пропусков

Производим проверку функцией isna() с суммированием пропусков

In [14]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [15]:
data_none = data.loc[data['total_income'].isna()]
data_none.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [18]:
data_none[data_none[index] == 72]['education']

NameError: name 'index' is not defined

Имеются 2174 отсутствующих значения в двух столбцах. Посмотрим на данные трудового стажа в 'days_employed'

In [11]:
data['days_employed'].head(15)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
Name: days_employed, dtype: float64

Колонка days_employed имеет невнятные данные с пропусками. Для решения поставленных вопросов эта колонка не понадобится. Удаляем.

In [12]:
field = 'days_employed'
try:
    data.drop([field], axis='columns', inplace=True)
    print('столбец', field, 'удален')
except:
    print('столбец', field, 'удален ранее')
data.columns

столбец days_employed удален


Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Как ты думаешь с чем связано наличие отрицательных значений ? 
    
Рекомендую не избавляться от используемых данных.
    
</div>

Колонка total_income:

In [13]:
pass_data = data['total_income'].isna().sum() / data['total_income'].count()
print('Пропусков в данных о доходах:', data['total_income'].isna().sum())
print('Пропуски составляют:{:.0%}'.format(pass_data))
print('Максимальный доход:', data['total_income'].max())
print('Минимальный доход :', data['total_income'].min())
print('Средний доход :', data['total_income'].mean())
median_income = data['total_income'].median()
print('Медиана ежемесячных доходов:', median_income)

Пропусков в данных о доходах: 2174
Пропуски составляют:11%
Максимальный доход: 2265604.028722744
Минимальный доход : 20667.26379327158
Средний доход : 167422.30220817294
Медиана ежемесячных доходов: 145017.93753253992


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Хорошо, что изучил стандартные статистические показатели.
    
---
    
У нас есть готовый метод `describe()`, который позволяет одним разом изучить все эти параметры.
    
</div>

Остались пропуски в солбце 'total_income'. Заменяем пропуски медианным значением. Среднее чувствительно к экстремальным значениям. 

In [14]:
data['total_income'].fillna(median_income, inplace=True)
#data.head(15)

Проверяем на пропуски повторно

In [15]:
print('Пропусков данных в ежемесячных доходах:', data['total_income'].isna().sum())

Пропусков данных в ежемесячных доходах: 0


### Вывод

Обнаружены пропущеные значения в двух столбцах: общий стаж и общий доход, в одних и тех-же строках. Вероятнее всего это ошибка выгрузки. В колонке трудового стажа данные не соответствуют реальным данным: Много отрицательных значений или значений соответствующим 500 лет и более. Столбец был удален полностью. В поле ежемесячного дохода отсутствующие значения были заменены медианой изза имеющихся экстремальных значений в верхнем диапазоне доходов.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Очень хорошо, что обратываем пропуски с помощью медианных значений и понимаем почему именно медианой. 
       
Давай попробуем сделать замену с помощью обработки по пропусков по группам то есть у нас есть информация по профессия, образованию. Поможет нам в этом методы `groupby()`, `trasform()` и `fillna()`.
    
Руководствоваться будем следующей логикой:
    
1. Создадим отдельный столбец, в который поместим медианное значение для каждого объекта исходя из группы профессий. В этом нам поможет метод `transform()`
2. Далее заменим пропуски используя полученный столбец
   
---
    
Таким образом мы сможем снизить вероятность сильного разброса в данных.
    
</div>

### Замена типа данных

Для лучшего восприятия заменим вещественный тип столбца 'total_income' на числовой

In [16]:
data['total_income'] = data['total_income'].astype('int')
#data.head()

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Вывод

Для замены типа данных использовался метод astype().  


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Как думаешь, для чего мы делаем замену типов в данных  ?    
</div>

### Обработка дубликатов

Посмотрим на дубликаты в таблице целиком методом duplicated(). просуммируем результат

In [18]:
print('Обнаружено дубликатов строк:', data.duplicated().sum())

Обнаружено дубликатов строк: 54


Удаляем дубликаты методом drop_duplicates() с параметром inplace=True для фактического удаления повторяющихся строк

In [19]:
data.drop_duplicates(inplace=True)

Проверяем дубликаты повторно

In [20]:
print('Обнаружено дубликатов строк:', data.duplicated().sum())

Обнаружено дубликатов строк: 0


Проверяем категориальные данные на дубликаты, приводим в порядок столбец 'education'

In [21]:
print(data['education'].value_counts())

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Значения находятся в различном регистре. Приводим к нижнему регистру методом  str.lower(). Проверяем результат.

In [22]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


### Вывод

Обнаружены дубликаты строк полностью, которые могли появиться по техническим причинам. Также Обработаны дубликаты с одним значением, с разным регистром, природа появления которых, вероятно на этапе сбора данных.

### Лемматизация

Для ответа на вопросы имеется необходимость лемматизировать колонку 'purpose' о целях получения кредита.

In [23]:
print(data['purpose'].value_counts())

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Изучив данные о целях получения кредита, значения можно разделить на 4 категории: 'свадьба' 'недвижимость' 'автомобиль' 'образование'. Добавим столбец лемм 'purpose_lemm' с категориями в таблицу с помощью функции, которая разберет списки лемм, объединенных методом join() от каждой строки на категории 'свадьба', 'недвижимость', 'автомобиль', 'образование'. Леммы "жильё" также отнесутся к 'недвижимость'.

In [24]:
def lemm_purpose(row):
    lemmas = ' '.join(m.lemmatize(row))
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif ('жилье' or 'недвижимость') in lemmas:        
        return 'недвижимость'    
    elif 'автомобиль' in lemmas:    
        return 'автомобиль'   
    elif 'образование' in lemmas:    
        return 'образование'
        
data['purpose_lemm'] = data['purpose'].apply(lemm_purpose)


Результат лемматизации в колонке 'purpose_lemm':

In [25]:
print(data['purpose_lemm'].value_counts())

недвижимость    4461
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose_lemm, dtype: int64


In [26]:
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Приведение строк к лемме проведено с помощью библиотеки pymystem3. mystem  это морфологический анализатор русского языка с поддержкой снятия морфологической неоднозначности. Т.е. все слова в строках приводятся в именительный падеж, единственное число, мужской род, а глаголы, причастия и деепричастия в инфинитиве несовершенного рода. Pymystem3 по умолчанию выдает список лемматизированных слов, который с помощью функции lemm_purpose(row) категоризируем и записывем в новое поле.

### Категоризация данных

#### Долг и дети

Создаем таблицу 'children_debt' с данными о детях и задолженностях.

In [27]:
children_debt = data.pivot_table('debt', index='children', aggfunc='sum') #Сводная таблица с суммой невозвратов 
children_debt['count_take'] = data['children'].value_counts() #Количество займов Всего
children_debt['non_return'] = children_debt['debt'] / children_debt['count_take']#Доля невозвратов
#children_debt

#### Долг и семейное положение

Создаем сводную таблицу: Индекс - 'family_status', колонки - данные из 'debt' .
вычисляем отношение должников к общему количеству

In [28]:
family_debt_pivot = data.pivot_table(index=['family_status'], values='family_status_id', columns= 'debt' ,aggfunc='count')
family_debt_pivot['ratio'] = (family_debt_pivot[1] / (family_debt_pivot[0] + family_debt_pivot[1]))
family_debt_pivot.head()

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,0.097509
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3775,388,0.093202
женат / замужем,11413,931,0.075421


#### Долг и доходы

Создадим категории дохода в таблице data в столбце 'income_level' по данным поля 'total_income': 
менее 90000 низкий доход;
от 90000 до 160000 - средний доход;
от 160000 до 250000 - выше среднего;
более 250000 - высоких доход

In [29]:
def income_level(row):
    income = row['total_income'] 
    if income >= 250000:
        return 'Высокий доход'
    elif 250000 > income >= 160000:
        return 'Выше среднего'
    elif 160000 > income >= 90000:
        return 'Средний доход'
    elif 90000 > income:
        return 'Низкий доход'
    
data['income_level'] = data.apply(income_level, axis=1)
#data['income_level'].value_counts()
#data.head()


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Расскажи пожалуйста каким образом получились данные уровни доходов ? 
    
</div>

Создаем сводную таблицу: Индекс - 'income_level', колонки - данные из 'debt' . вычисляем отношение должников к общему количеству

In [30]:
income_pivot = data.pivot_table(index=['income_level'], values='total_income', columns= 'debt' ,aggfunc='count')
income_pivot['ratio'] = (income_pivot[1] / (income_pivot[0] + income_pivot[1]))
#income_pivot

#### Долг и цели

Создадим сводную таблицу по мемметизированным целям кредита, добавим долю задолженности в последнем столбце

In [31]:
purpose_pivot = data.pivot_table(index=['purpose_lemm'], values='total_income', columns= 'debt' ,aggfunc='count')
purpose_pivot['ratio'] = (purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1]))
#purpose_pivot

### Вывод

Данные подготовлены для ответов на поставленные вопросы. По каждой категории составлены таблицы для удобного отображения.
Объединенины избранные данные в произвольныx группах по заданному критерию.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Обрати пожалуйста внимание, что мы выполняем повторяющиеся действия. Это означает — оптимизация.

То есть мы можем сделать например функцию, которая будет выполнять все наши расчеты и выводить необходимые нам результаты.

</div>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
childfree = children_debt.loc[0, 'non_return']
print('Невозврат кредитов в срок среди бездетных {:.2%}'.format(childfree))
inchild = children_debt.loc[1:, 'debt'].sum() / children_debt.loc[1:, 'count_take'].sum()
print('Невозврат кредитов в срок среди родителей {:.2%}'.format(inchild))

Невозврат кредитов в срок среди бездетных 7.53%
Невозврат кредитов в срок среди родителей 9.24%


### Вывод

Невозврат кредитов в срок среди заемщикоов, имеющих детей выше.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
family_debt_pivot.sort_values(by = 'ratio', ascending = True)

debt,0,1,ratio
family_status,,,
вдовец / вдова,896,63,0.065693
в разводе,1110,85,0.071130
женат / замужем,11413,931,0.075421
гражданский брак,3775,388,0.093202
Не женат / не замужем,2536,274,0.097509


### Вывод

Имеется зависимость невозврата. Те, кто был в браке более ответственно относятся к возврату кредита


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
income_pivot.sort_values(by = 'ratio', ascending = True)

debt,0,1,ratio
income_level,,,
Высокий доход,2618,194,0.068990
Низкий доход,3086,262,0.078256
Выше среднего,4962,444,0.082131
Средний доход,9064,841,0.084907


### Вывод

Имеется зависимость. Заемщики с высоким и низким доходом отдают кредиты лучше


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
purpose_pivot.sort_values(by = 'ratio', ascending = True)

debt,0,1,ratio
purpose_lemm,,,
недвижимость,4153,308,0.069043
свадьба,2149,186,0.079657
образование,3644,370,0.092177
автомобиль,3905,403,0.093547


### Вывод

Наиболее надежные кредиты - кредиты на недвижимость


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

### Шаг 4. Общий вывод

<div class="alert alert-danger">
<h2> Комментарий ревьюера</h2>

Финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. В нем можно приводить полученные в ходе работы значения. Также можно расписать все, что было сделано в работе.
    
Самое главное — это вывод должен содержать в себе ответ на поставленную задачу.

</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.